# Extension Project 
## Part 1: Extending the Analysis of AGN to include the X-ray band.
This script is intended to provide an extension of the work done in the first script. This extension in it's current form as no real aim and is intended to be exploratory. 


Notes for Project Extension 
* Finish off by fixing the Donley Selection
    * If time permits add the Stern Selection
* Maybe crossmatch with X-Ray data?
    * Focus on one field, CDFS with XMM-Newton data
    * chandra space 4Ms or 7Ms survey in topcat
    * Make liberal use of the crossmatching feature in TOPCAT or similarly follow on with this project and use the crossmatching feature in python
    * Use the XMM-Newton data to find the X-ray fluxes of the AGN
    * Use the seleciton criteria in Cowley et al. 2016 to select AGN by Luminosity and Hardness Ratio.
* Xray selection
* AGN Selection using luminosity threshold (CDFS)
* Possible UVJ selection
    * Potentially contaiminated by dusty torus
    * What we could do would be to first plot the UVJ diagram and see how the distribution looks in different redshift bins
    * Then we could plot the UVJ diagram for the AGN selected by the X-ray luminosity and see how that compares to the general population
    * we could also compare this to the IR selected AGNS, maybe using a subset of these AGN that are also X-ray selected.
* Could also look at how star-formation rate is effected between xray agn or ir agn?


HR <- Crossmatch the the HR from the XRAY ZFourge catalogue.



# Cross Matching ZFOURGE with Chandra Data from the 4Ms Survey
* Focus on one field, CDFS with Chandra Data
* chandra space 4Ms survey in topcat (Check Paper)
    * Use the 4Ms CDFS Survey data to find the X-ray luminosities of the AGN, HR, etc.
* Make liberal use of the crossmatching feature in TOPCAT or similarly follow on with this project and create a cross-matching function in python
* Use the seleciton criteria in Cowley et al. 2016 to select AGN by Luminosity and Hardness Ratio.

In [2]:
# Begin by importing the required packages for the project
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
from astropy.io import fits
import os
import astropy.units as u
from astropy.coordinates import SkyCoord

In [3]:
# Attempt to read in the fits file of the CDFS 7Ms survey or the Chandra 4Ms Survey.
# Noting for the 7Ms survey the the there is a flux but not a value for lumunosity.

# For now we will use the CDFS 4Ms survey (Xue et al. 2011)
# importing the fits file and convert to a dataframe
file_path = os.path.join('crossmatch\CDFS_4Ms.fits')
xray_fits = fits.open(file_path)
xray_df = pd.DataFrame(np.array(xray_fits[1].data).byteswap().newbyteorder())  # Byteswap so that Pandas can read it

## check the df
a = xray_df['RAJ2000'].iloc[0]
str(a.decode()) # decode the byte string, this will need to be done for everything that is a byte string



'03 31 35.79'

In [4]:
# We now want to read in the CDFS field from the ZFOURGE survey catalogue and do a cross match
# on this particular dataset

catalog_file = os.path.join('cdfs/cdfs.v1.6.11.cat')
cdfs_df = pd.DataFrame(np.genfromtxt(catalog_file, names=True, dtype=None, encoding='ascii'))
cdfs_df.head()

,id,x,y,ra,dec,SEflags,iso_area,fap_Ksall,eap_Ksall,apcorr,...,wmin_jhk,wmin_hst,wmin_irac,wmin_all,star,nearstar,use,snr,use_nosnr,z_spec
0,1,4078.527,203.090,53.100105,-27.966728,19,5484.0,73.393000,0.263729,0.996778,...,0.0,0.0,1330.8,0.1,0,0,0,251.5565,0,-99.0
1,2,6292.975,55.927,52.995632,-27.972803,2,315.0,9.561428,0.251456,1.058802,...,0.0,0.0,0.0,0.0,0,0,0,41.6570,0,-99.0
2,3,1791.772,51.199,53.207989,-27.973034,2,375.0,10.821210,0.263729,1.048642,...,0.0,0.0,0.0,0.0,0,0,0,44.3473,0,-99.0
3,4,4181.315,176.410,53.095257,-27.967840,3,1060.0,76.361450,0.263729,1.001701,...,0.0,0.0,0.0,0.0,0,0,0,276.7351,0,-99.0
4,5,5997.950,48.860,53.009548,-27.973110,0,219.0,26.528060,0.251456,1.097179,...,0.0,0.0,0.0,0.0,0,0,0,155.9100,0,-99.0


In [16]:
print(xray_df.columns)

Index(['recno', 'n_ID', 'ID', 'RAJ2000', 'DEJ2000', 'ePos', 'log(P)',
       'log(Pm)', 'OffAng', 'Fcts', 'E_Fcts', 'e_Fcts', 'Scts', 'E_Scts',
       'e_Scts', 'Hcts', 'E_Hcts', 'e_Hcts', 'Fext', 'RAC', 'DEC', 'C-off',
       'Cmag', 'Nx', 'Ccat', 'RAW', 'DEW', 'Wmag', 'RAGD', 'DEGD', 'GDmag',
       'RAG', 'DEG', 'Gmag', 'RAM', 'DEM', 'Mmag', 'RAMY', 'DEMY', 'MYmag',
       'RAS', 'DES', 'Smag', 'RAV', 'DEV', 'Vmag', 'zsp', 'q_zsp', 'r_zsp',
       'zphl', 'b_zphl', 'B_zphl', 'zphla', 'zphc', 'b_zphc', 'B_zphc',
       'q_zphc', 'zphr', 'b_zphr', 'B_zphr', 'zadopt', 'L8', 'RAL8', 'DEL8',
       'L5', 'RAL5', 'DEL5', 'FExp', 'SExp', 'HExp', 'BRat', 'E_BRat',
       'e_BRat', 'PInd', 'EPInd', 'ePInd', 'FFlux', 'SFlux', 'HFlux', 'Lx',
       'OType', 'Notes', 'Simbad'],
      dtype='object')
